In [61]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [62]:
# filepath_prefix = "../data/data2/seg_train/seg_train/"
# 
# buildings = load_class_data(f"{filepath_prefix}buildings")
# street = load_class_data(f"{filepath_prefix}street")
# forest = load_class_data(f"{filepath_prefix}forest")
# glacier = load_class_data(f"{filepath_prefix}glacier")
# sea = load_class_data(f"{filepath_prefix}sea")
# mountain = load_class_data(f"{filepath_prefix}mountain")

In [63]:
# x = np.concatenate([buildings, street, forest, glacier, sea, mountain])
# y = np.concatenate(
#     [
#         [0] * buildings.shape[0],
#         [1] * street.shape[0],
#         [2] * forest.shape[0],
#         [3] * glacier.shape[0],
#         [4] * sea.shape[0],
#         [5] * mountain.shape[0],
#     ]
# )

In [39]:
N_EPOCHS = 50
N_TRIALS = 5
BATCH_SIZE = 128

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [40]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)

x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [41]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

assessment_model.fit(x_train, y_train, epochs=30, batch_size=BATCH_SIZE)

Epoch 1/30
63/63 [==============================] - 1s 15ms/step - loss: 2.4078 - accuracy: 0.1601
Epoch 2/30
63/63 [==============================] - 1s 14ms/step - loss: 2.0998 - accuracy: 0.2285
Epoch 3/30
63/63 [==============================] - 1s 15ms/step - loss: 1.9350 - accuracy: 0.2895
Epoch 4/30
63/63 [==============================] - 1s 16ms/step - loss: 1.8487 - accuracy: 0.3256
Epoch 5/30
63/63 [==============================] - 1s 15ms/step - loss: 1.7816 - accuracy: 0.3426
Epoch 6/30
63/63 [==============================] - 1s 16ms/step - loss: 1.7182 - accuracy: 0.3626
Epoch 7/30
63/63 [==============================] - 1s 15ms/step - loss: 1.6925 - accuracy: 0.3770
Epoch 8/30
63/63 [==============================] - 1s 15ms/step - loss: 1.6318 - accuracy: 0.3879
Epoch 9/30
63/63 [==============================] - 1s 14ms/step - loss: 1.6051 - accuracy: 0.4065
Epoch 10/30
63/63 [==============================] - 1s 15ms/step - loss: 1.5627 - accuracy: 0.4223
Epoch 11/

In [42]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]

y_pred_assessment = assessment_model.predict(x_train_sorted, verbose=0)
losses_assessment = - np.array(utils.calculate_loss_per_sample(y_train_sorted, y_pred_assessment, loss))

In [45]:
_, counts = np.unique(y_train_sorted, return_counts=True)

losses_proba = utils.normalize_losses_per_group(counts, losses_assessment)
losses_proba2 = np.exp(losses_proba) / sum(np.exp(losses_proba))

In [53]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [54]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
            
            samples_ids = np.random.choice(range(train_size), size=n_samples, replace=False)
            # samples_ids = np.random.choice(range(train_size), p=losses_proba2, size=n_samples, replace=False)
            
            model.fit(
                x_train_sorted[samples_ids], y_train_sorted[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE, verbose=1
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


  0%|          | 0/5 [00:00<?, ?it/s]

8/8 [==============================] - 0s 9ms/step - loss: 1.3353 - accuracy: 0.5180


 20%|██        | 1/5 [01:44<06:57, 104.38s/it]

8/8 [==============================] - 0s 8ms/step - loss: 1.3288 - accuracy: 0.5110


...

 60%|██████    | 3/5 [04:51<03:11, 95.92s/it]

8/8 [==============================] - 0s 8ms/step - loss: 1.6686 - accuracy: 0.4140


 80%|████████  | 4/5 [06:26<01:35, 95.56s/it]

8/8 [==============================] - 0s 8ms/step - loss: 1.3451 - accuracy: 0.5080


100%|██████████| 5/5 [07:58<00:00, 95.67s/it]


In [56]:
results

{'test_model_1': [0.5180000066757202,
  0.5109999775886536,
  0.5189999938011169,
  0.414000004529953,
  0.5080000162124634]}

In [57]:
sum(results["test_model_1"]) / 5

0.49399999976158143

In [ ]:
{'test_model_1': [0.5270000100135803,
  0.5080000162124634,
  0.5299999713897705,
  0.5419999957084656,
  0.4309999942779541]}
0.5075999975204468

In [51]:
sum(results["test_model_1"]) / 5

0.5075999975204468

In [ ]:
{'test_model_1': [0.5070000290870667,
  0.5040000081062317,
  0.527999997138977,
  0.41499999165534973,
  0.5329999923706055]}

In [ ]:
0.4974000036716461